#### Read swot l2 pixel cloud data and write out to masked file.


In [9]:
import os
import xarray as xr
from glob import glob
import geopandas as gpd
from utils.swot_data_mask import swot_pixc_mask, swot_raster_mask
from utils.functions import swot_raster_reproj


In [10]:
dir_pixc = 'data/dianchi-lake/pixc'
dir_raster = 'data/dianchi-lake/raster'
path_lake_vec = 'data/dianchi-lake/dianchi_vector.gpkg'


In [11]:
## read vector file of the lake.
lake_gpd = gpd.read_file(path_lake_vec)
lake_gpd


,Name,geometry
0,dianchi,"POLYGON Z ((102.66614 24.83805 0.00000, 102.65..."


#### 1. SWOT pixel cloud: batch data read, mask and write out

In [12]:
## Check original .nc raster file.
paths_pixc_file = [os.path.join(dir_pixc, file) for file in os.listdir(dir_pixc) if 'PIXC_' in file and 'masked' not in file]
paths_pixc_file = sorted(paths_pixc_file)
print(len(paths_pixc_file))


9


In [13]:
for i, path in enumerate(paths_pixc_file):
  print(f'input file: {i} {path}')
  # Define the output path
  path_masked = path.split('.')[0]+'_masked.nc'
  pixc_nc = xr.open_dataset(path, group='pixel_cloud')
  pix_vars = swot_pixc_mask(pixc_nc=pixc_nc, region_gpd=lake_gpd, path_masked=path_masked)


input file: 0 data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_009_049_198L_20240105T211648_20240105T211659_PIC0_01.nc
file read and written to data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_009_049_198L_20240105T211648_20240105T211659_PIC0_01_masked.nc
input file: 1 data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_010_049_198L_20240126T180154_20240126T180205_PIC0_01.nc
file read and written to data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_010_049_198L_20240126T180154_20240126T180205_PIC0_01_masked.nc
input file: 2 data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_012_049_198L_20240308T113203_20240308T113214_PIC0_01.nc
file read and written to data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_012_049_198L_20240308T113203_20240308T113214_PIC0_01_masked.nc
input file: 3 data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_014_049_198L_20240419T050213_20240419T050224_PIC0_01.nc
file read and written to data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_014_049_198L_20240419T050213_20240419T050224_PIC0_01_masked.nc
input file: 4 data/dianchi-lake/pixc/SWOT_L2_HR_PIXC_016

#### 2. SWOT raster data: batch data read, mask and write out

In [14]:
## Check original .nc raster file.
paths_raster_file = [os.path.join(dir_raster, file) for file in os.listdir(dir_raster) 
                                            if 'Raster_' in file and 'masked' not in file]
paths_raster_file = sorted(paths_raster_file)
print(len(paths_raster_file))


9


In [15]:
for i, path in enumerate(paths_raster_file):
    print(f'input file: {i} {path}')
    # Define the output path
    path_masked = path.split('.')[0]+'_masked.nc'
    raster_nc = xr.open_dataset(path, decode_times=False)
    raster_epsg = '326' + str(raster_nc.attrs['utm_zone_num'])
    raster_wgs84_nc = swot_raster_reproj(raster_nc, epsg_from=raster_epsg, epsg_to='4326')
    raster_vars = swot_raster_mask(raster_nc=raster_wgs84_nc, 
                                region_gpd=lake_gpd, path_masked=path_masked)


input file: 0 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_009_049_099F_20240105T211638_20240105T211659_PIC0_01.nc
file read and written to data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_009_049_099F_20240105T211638_20240105T211659_PIC0_01_masked.nc
input file: 1 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_010_049_099F_20240126T180144_20240126T180205_PIC0_01.nc
file read and written to data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_010_049_099F_20240126T180144_20240126T180205_PIC0_01_masked.nc
input file: 2 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_012_049_099F_20240308T113153_20240308T113214_PIC0_01.nc
file read and written to data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_012_049_099F_20240308T113153_20240308T113214_PIC0_01_masked.nc
input file: 3 data/dianchi-lake/raster/SWOT_L2_HR_Raster_100m_UTM48R_N_x_x_x_014_049_099F_20240419T050203_20240419T050224_PIC0_01.nc
file read and w